In [1]:
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import box
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from shapely.geometry import Point
from pyproj import Proj, transform, CRS

In [2]:
import matplotlib.font_manager as fm

# 한국어 출력을 위한 폰트 설정
plt.rc('font', family='NanumGothic') 

# 마이너스 기호가 깨지는 것을 방지
plt.rcParams['axes.unicode_minus'] = False

In [3]:
data = gpd.read_file("data/서울시 비상소화장치 위치정보.csv")

In [4]:
data

,연번,비상소화장치일련번호,일련번호,X좌표,Y좌표,비상소화장치고유번호,사용구분,삭제여부,업데이트날짜,geometry
0,1,110080030,451,192436.05620538,552336.95498254,41,52201,N,20231116120041,None
1,2,110080031,452,191972.54288665,554384.19107578,54,52202,N,20231017120007,None
2,3,110080032,453,195705.68084358,555164.49557405,56,52201,N,20231111120009,None
3,4,110080033,454,195646.35573731,555661.51678319,57,52201,N,20231111120009,None
4,5,110080034,455,191543.20535296,555165.58938338,55,52201,N,20231024120006,None
...,...,...,...,...,...,...,...,...,...,...
2690,2691,110086927,0,195867.26344686,550812.72472093,000118,52201,N,20231115120002,None
2691,2692,110086928,0,195978.50137131,550812.36698320,000119,52201,N,20231109120021,None
2692,2693,110086929,0,196051.06076163,550814.24186619,000120,52201,N,20231109120021,None
2693,2694,110086930,0,196029.38712340,550750.44879355,000121,52201,N,20231109120021,None


In [5]:
# 좌표계 정의
# 한국 중부원점(Korea 2000 Central Belt) EPSG:5186
proj_korea = Proj(init='epsg:5186')
# WGS84
proj_wgs84 = Proj(init='epsg:4326')

C:\Users\1qlqj\Desktop\multicamp_semi\multicamp_semi\venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\1qlqj\Desktop\multicamp_semi\multicamp_semi\venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [6]:
# X좌표와 Y좌표를 위도와 경도로 변환하는 함수
def convert_coords(x, y):
    lon, lat = transform(proj_korea, proj_wgs84, x, y)
    return lat, lon

In [7]:
# 모든 좌표 변환
data[['위도', '경도']] = data.apply(lambda row: convert_coords(row['X좌표'], row['Y좌표']), axis=1, result_type="expand")

C:\Users\1qlqj\AppData\Local\Temp\ipykernel_7300\3974450290.py:3: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(proj_korea, proj_wgs84, x, y)


KeyboardInterrupt: 

In [ ]:
data

In [ ]:
# Point 객체로 변환
geometry = [Point(xy) for xy in zip(data["경도"], data["위도"])]

In [ ]:
# GeoDataFrame 생성
gdf = gpd.GeoDataFrame(data, geometry=geometry)

In [ ]:
gdf = gdf.drop(columns = ['일련번호', 'X좌표', 'Y좌표', '사용구분', '삭제여부'])

In [ ]:
gdf = gdf.drop(columns = ['업데이트날짜', '위도', '경도'])

In [ ]:
gdf.head()

In [ ]:
# gdf.to_csv('서울시_비상소화장치_좌표.csv', index=False)

In [ ]:
dong = pd.read_file("data/동경계/동경계_geo.shp")

In [ ]:
dong